In [ ]:
# pip install pandasql

In [ ]:
import pandas as pd
import sqlalchemy


In [ ]:
df = pd.read_csv(r'C:\Users\kaua_\OneDrive\Documentos\Bootcamp_xp\arquiteto_big_data\desafio_final\dados_licencas_medicas.csv', sep=';', encoding='latin1')
df = df.drop_duplicates()
df = df.dropna()
df.count()

In [ ]:
user= 'root'
password= ''
host= 'localhost'
database= 'coleta_df'
string_conexao= f'mysql://{user}:{password}@{host}/{database}'
string_conexao

engine = sqlalchemy.create_engine(string_conexao)
conn= engine.connect()



In [ ]:
df

In [ ]:

colunas_esp= df['especialidade'].unique()
colunas_esp= pd.DataFrame(colunas_esp, columns=['especialidade'])
colunas_esp

colunas_esp.to_sql('tb_especialidade',conn,if_exists='append', index=False)
    

In [ ]:
coluna_lic = df['motivo_licença'].unique()
coluna_lic= pd.DataFrame(coluna_lic,columns=['motivo_licenca'])
coluna_lic

coluna_lic.to_sql('tb_motivo_licenca',conn,if_exists='append',index=False)

In [ ]:
df_esp=pd.read_sql('select * from tb_especialidade', conn)
df_esp

In [ ]:
# test= """ select distinct a.nome_medico, b.cod_especialidade
            
#             from df a
            
#             left join df_esp b
#             on b.especialidade=a.especialidade
# """


df_med = pd.merge(df[['nome_medico', 'especialidade']], 
                  df_esp[['especialidade', 'cod_especialidade']], 
                  on='especialidade', 
                  how='left')

df_med= df_med.drop_duplicates().reset_index()
coluna_med= df_med[['nome_medico' , 'cod_especialidade']]
coluna_med


In [ ]:
coluna_med.to_sql('tb_medico',conn,if_exists='append',index=False)


In [ ]:
coluna_est = df['estado_colaborador'].unique()
coluna_est= pd.DataFrame(coluna_est,columns=['estado'])
coluna_est

coluna_est.to_sql('tb_estado',conn,if_exists='append',index=False)

In [ ]:
tb_estado_civil = pd.read_sql('select * from tb_estado_civil', conn)
tb_escolaridade = pd.read_sql('select * from tb_escolaridade',conn)
tb_hobbie = pd.read_sql('select * from tb_hobbie', conn)
tb_estado = pd.read_sql('select * from tb_estado', conn)

In [ ]:
coluna_est_civil = df['estado_civil'].unique()
coluna_est_civil= pd.DataFrame(coluna_est_civil,columns=['estado_civil'])

coluna_est_civil

coluna_est_civil.to_sql('tb_estado_civil',conn, if_exists='append', index=False)

In [ ]:
coluna_esc = df['escolaridade'].unique()
coluna_esc = pd.DataFrame(coluna_esc,columns=['escolaridade'])

coluna_esc.to_sql('tb_escolaridade',conn,if_exists='append',index=False)

In [ ]:
coluna_hobbie = df['hobbie'].unique()
coluna_hobbie = pd.DataFrame(coluna_hobbie,columns=['hobbie'])

coluna_hobbie.to_sql('tb_hobbie', engine, if_exists='append', index=False)

In [ ]:

merged_df = pd.merge(df, tb_estado_civil, left_on='estado_civil', right_on='estado_civil', how='left')
merged_df = pd.merge(merged_df, tb_escolaridade, left_on='escolaridade', right_on='escolaridade', how='left')
merged_df = pd.merge(merged_df, tb_hobbie, left_on='hobbie', right_on='hobbie', how='left')
merged_df = pd.merge(merged_df, tb_estado, left_on='estado_colaborador', right_on='estado', how='left')

result = merged_df[['nome_colaborador', 'data_nascimento', 'sexo_colaborador', 'salario', 'qtd_filhos', 'possui_pet', 'cod_estado_civil', 'cod_escolaridade', 'cod_hobbie', 'cod_estado']]

result = result.drop_duplicates()

result.to_sql('tb_colaborador',conn,if_exists='append',index=False)


In [ ]:
esp = pd.read_sql('select * from tb_especialidade',conn)
lic = pd.read_sql('select * from tb_motivo_licenca',conn)
colaborador = pd.read_sql('select * from tb_colaborador',conn)
colaborador


In [ ]:

merged_df = df.merge(esp, how='left', left_on='especialidade', right_on='especialidade')

merged_df = merged_df.merge(lic, how='left', left_on='motivo_licença', right_on='motivo_licenca')

merged_df = merged_df.merge(colaborador, how='left', left_on='nome_colaborador', right_on='nome_colaborador')

merged_df.rename(columns={'duracao_licensa': 'duracao_licenca'}, inplace=True)

colunas_desejadas = ['data_processamento', 'inicio_licenca', 'fim_licenca', 'duracao_licenca',
                     'cod_especialidade', 'cod_motivo_licenca', 'cod_colaborador']


for coluna in colunas_desejadas:
    if coluna not in merged_df.columns:
        merged_df = merged_df.assign(**{coluna: None})

result = merged_df[colunas_desejadas]

result.to_sql('tb_processamento_licenca',conn, if_exists='append', index=False)

